In [21]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
import skimage.transform

In [24]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    modality = ds.Modality 
    if modality !=  'DX':
        print('Modality is not DX. The image will not be assessed.')
        print()
        return None

    body_part = ds.BodyPartExamined
    if body_part !=  'CHEST':
        print('Body part examined is not Chest. The image will not be assessed.')
        print()
        return None
    
    patient_position= ds.PatientPosition 
    if patient_position not in ['PA', 'AP']:
        print('Patient position is not PA or AP. The image will not be assessed.')
        print()
        return None
    
    findings = ds.StudyDescription
    print('Findings:', findings)
    plt.imshow(img, cmap='gray')
    plt.show()
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
    proc_img = (img-img_mean) / img_std
    proc_img = skimage.transform.resize(proc_img, img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):

    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
        model = model_from_json(model_json)
        model.load_weights(weight_path)    
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_Y = model.predict(img)   
    if pred_Y > thresh:
        return 'Pneumonia'
    else:
        return 'Not pneumonia'
    
    return prediction

In [25]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "/Users/mostafazaky/Project 1/model.json"
weight_path = "{}_model.best.hdf5".format('xray_class')

IMG_SIZE=(1,224,224,3) 
img_mean = 0
img_std = 255

my_model = load_model(model_path, weight_path)
thresh = 0.4035850763320923

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

TypeError: Could not locate class 'Sequential'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'class_name': 'Sequential', 'config': {'name': 'sequential_1', 'layers': [{'class_name': 'Model', 'config': {'name': 'model_1', 'layers': [{'name': 'input_1', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_1'}, 'inbound_nodes': []}, {'name': 'block1_conv1', 'class_name': 'Conv2D', 'config': {'name': 'block1_conv1', 'trainable': False, 'filters': 64, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regulari